In [1]:
import os
import tensorflow as tf
from PIL import Image
import PIL
import numpy as np

2025-06-13 16:03:41.431895: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-13 16:03:41.451530: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-13 16:03:41.484728: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749805421.509629   20974 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749805421.517633   20974 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749805421.542652   20974 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
folder_path = os.getcwd()
ws_path = os.path.dirname(folder_path)
ws_path

'/home/kadafi/Documents/AWU/Heart-Leaks-Disease-Classification'

In [3]:
list_dir = os.listdir(ws_path)
frame_path = ws_path + '/rawFrame'
prepro_data_path = ws_path + '/preproDataset'
nor_fr_path = frame_path + '/normal'
vsd_fr_path = frame_path + '/vsd'
nor_prepro_path = prepro_data_path + '/normal'
vsd_prepro_path = prepro_data_path + '/vsd'

In [4]:
# Create output directory
os.makedirs(prepro_data_path, exist_ok=True)
os.makedirs(nor_prepro_path, exist_ok=True)
os.makedirs(vsd_prepro_path, exist_ok=True)

In [7]:
nor_img = os.listdir(nor_fr_path)
nor_fr_list_path = list()

for img in nor_img:
    path = nor_fr_path + f'/{img}'
    nor_fr_list_path.append(path)

# nor_fr_list_path

In [8]:
vsd_img = os.listdir(vsd_fr_path)
vsd_fr_list_path = list()

for img in vsd_img:
    path = vsd_fr_path + f'/{img}'
    vsd_fr_list_path.append(path)

# vsd_fr_list_path

In [18]:
import pandas as pd


df = pd.DataFrame({'path': nor_fr_list_path, 'label': 0})
df = pd.concat([df, pd.DataFrame({'path': vsd_fr_list_path, 'label': 1})])
x = df['path']
y = df['label']

In [20]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [30]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
x_res, y_res = rus(x, y)
x_res

ImportError: cannot import name 'is_scalar_nan' from 'sklearn.utils' (/home/kadafi/Documents/AWU/Heart-Leaks-Disease-Classification/venv/lib/python3.12/site-packages/sklearn/utils/__init__.py)

In [12]:
def preprocess(input_path, output_path, img_size=(224, 224)):
    for path in input_path:
        img = tf.keras.preprocessing.image.load_img(path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)

        img_array = img_array / 255.0
        img_uint8 = tf.image.convert_image_dtype(img_array, dtype=tf.uint8)

        filename = os.path.basename(path)
        filename = os.path.splitext(filename)[0] + '.png'

        output_file = os.path.join(output_path, filename)

        tf.keras.utils.save_img(output_file, img_uint8)


In [13]:
preprocess(nor_fr_list_path, nor_prepro_path)
preprocess(vsd_fr_list_path, vsd_prepro_path)